In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import nltk 
from time import time
from collections import Counter
from nltk.corpus import stopwords
from nltk import FreqDist,ngrams,word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import PyPDF2 

/usr/local/lib/python2.7/dist-packages/pandas/_libs/__init__.py:4: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
/usr/local/lib/python2.7/dist-packages/pandas/__init__.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (hashtable as _hashtable,
/usr/local/lib/python2.7/dist-packages/pandas/core/dtypes/common.py:6: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import algos, lib
/usr/local/lib/python2.7/dist-packages/pandas/core/util/hashing.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import hashing, tslib
/usr/local/lib/python2.7/dist-packages/pandas/core/indexes/base.py:7: RuntimeWarning: numpy.dtype size changed, may indicate bi

In [54]:
# creating a pdf file object
pdfFileObj = open('Intro_to_ir_manning.pdf', 'rb')
 
# creating a pdf reader object
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
 
# printing number of pages in pdf file
print(pdfReader.numPages)

num_pages = pdfReader.numPages
count = 0
text = ""
#The while loop will read each page
while count < 30:
    pageObj = pdfReader.getPage(count)
    count +=1
    text += pageObj.extractText()
print text
pdfFileObj.close()

581
An
Introduction
to
Information
Retrieval
DraftofApril1,2009
An
Introduction
to
Information
Retrieval
ChristopherD.Manning
PrabhakarRaghavan
HinrichSchütze
CambridgeUniversityPress
Cambridge,England
DRAFT!

DONOTDISTRIBUTEWITHOUTPRIORPERMISSION

©
2009CambridgeUniversityPress
ByChristopherD.Manning,PrabhakarRaghavan&HinrichSch
ütze
PrintedonApril1,2009

Website:http://www.informationretrieval.org/

Comments,corrections,andotherfeedbackmostwelcomeat:

informationretrieval@yahoogroups.com
DRAFT!
©
April1,2009CambridgeUniversityPress.Feedbackwelcome
.
v
BriefContents

1Booleanretrieval
1
2Thetermvocabularyandpostingslists
19
3Dictionariesandtolerantretrieval
49
4Indexconstruction
67
5Indexcompression
85
6Scoring,termweightingandthevectorspacemodel
109
7Computingscoresinacompletesearchsystem
135
8Evaluationininformationretrieval
151
9Relevancefeedbackandqueryexpansion
177
10XMLretrieval
195
11Probabilisticinformationretrieval
219
12Languagemodelsforinformationretrieval
237
13Textclassic

In [3]:
def remove_stopwords(data):
    stop_words = Counter(stopwords.words('english'))
    ans = []
    for each in data:
        if(each not in stop_words.keys()):
            ans.append(each)
    return ans

In [4]:
def lemmatizer(data):
    lmtzr = WordNetLemmatizer()
    ans = []
    for each in data:
        ans.append(lmtzr.lemmatize(each))
    return ans

In [5]:
def chapterwise(text):
    tokens = word_tokenize(text[0:2000])
    tokens = remove_stopwords(tokens)
    tokens = lemmatizer(tokens)
        
    fDist = FreqDist(tokens)
    dictionary_freq = dict(zip(fDist.keys(),fDist.values()))
    sorted_dict_freq = sorted(dictionary_freq.items(), key=lambda x: x[1],reverse= True)
    
    return dict(sorted_dict_freq)

In [33]:
def book(dataset, top_k = 20, num_chap = 10, ch = False):
    data = pd.read_csv(dataset)
    
    if ch : 
        for i in range(num_chap):
            text = data['Text'][i]
        
            tokens = word_tokenize(text[0:2000])
            tokens = remove_stopwords(tokens)
            tokens = lemmatizer(tokens)

            fDist = FreqDist(tokens)
            dictionary_freq = dict(zip(fDist.keys(),fDist.values()))
            sorted_dict_freq = sorted(dictionary.items(), key=lambda x: x[1],reverse= True)
        
    else :
        text = ""
        for i in range(data.shape[0]):
            text += data['Text'][i]
        text = text.decode('utf-8')

        tokens = word_tokenize(text)
        tokens = remove_stopwords(tokens)
        tokens = lemmatizer(tokens)

        # Frequency
        fDist = FreqDist(tokens)
        dictionary_freq = dict(zip(fDist.keys(),fDist.values()))
        sorted_dict_freq = sorted(dictionary_freq.items(), key=lambda x: x[1],reverse= True)

        print "Top "+str(top_k)+" Words according to frequency: "
        for i in range(top_k): 
            print str(i+1)+". "+sorted_dict_freq[i][0]," : ",sorted_dict_freq[i][1]


        # entropy
        total = float(len(tokens))
        dictionary_entropy = {k: -1 * (float(freq) / total) * np.log(float(freq)/total) for k, freq in dictionary_freq.iteritems()} 
        sorted_dict_entropy = sorted(dictionary_entropy.items(), key=lambda x: x[1],reverse= True)

        print "\n\nTop "+str(top_k)+" Words according to entropy: "
        for i in range(top_k): 
            print str(i+1)+". "+sorted_dict_entropy[i][0]," : ",sorted_dict_entropy[i][1]
        
        #own metric
        freq_own = np.zeros(( len(dictionary_freq), num_chap))
        index_tokens = {k: i for i,k in enumerate(dictionary_freq)}
        
        for i, chap in data.iterrows():        
            if i>=num_chap : break
            freq_dict_chapterwise = chapterwise(chap['Text'])
            for k,v in freq_dict_chapterwise.iteritems():
                freq_own[index_tokens[k]][i] = v

        stand_dev = np.std(freq_own, axis = 1)
        std_dict = {k: stand_dev[index_tokens[k]] for k in index_tokens}
        sorted_dict_std = sorted(std_dict.items(), key=lambda x: x[1],reverse= False)


        print "\n\nTop "+str(top_k)+" Words according to our metric: "
        for i in range(top_k): 
            print str(i+1)+". "+sorted_dict_std[i][0]," : ",sorted_dict_std[i][1]
        

In [35]:
book('questions-data.csv',top_k = 200, num_chap= 10)

KeyboardInterrupt: 

In [36]:
data = pd.read_csv('questions-data.csv')
top_k = 200
num_chap = 10
text = ""
for i in range(data.shape[0]):
    text += data['Text'][i]
text = text.decode('utf-8')

tokens = word_tokenize(text)
tokens = remove_stopwords(tokens)
tokens = lemmatizer(tokens)

# Frequency
fDist = FreqDist(tokens)
dictionary_freq = dict(zip(fDist.keys(),fDist.values()))
sorted_dict_freq = sorted(dictionary_freq.items(), key=lambda x: x[1],reverse= True)


In [43]:
 #own metric
freq_own = np.zeros(( len(dictionary_freq), num_chap))
index_tokens = {k: i for i,k in enumerate(dictionary_freq)}

for i, chap in data.iterrows():        
    if i>=num_chap : break
    freq_dict_chapterwise = chapterwise(chap['Text'])
    for k,v in freq_dict_chapterwise.iteritems():
        freq_own[index_tokens[k]][i] = v

stand_dev = np.std(freq_own, axis = 1)
std_dict = {k: stand_dev[index_tokens[k]] for k in index_tokens}
sorted_dict_std = sorted(std_dict.items(), key=lambda x: x[1],reverse= True)

In [45]:
print "\n\nTop "+str(top_k)+" Words according to our metric: "
for i in range(top_k): 
    print str(i+1)+". "+sorted_dict_std[i][0]," : ",sorted_dict_std[i][1]




Top 200 Words according to our metric: 
1. course  :  1.5
2. ?  :  1.4177446878757827
3. ,  :  1.268857754044952
4. tool  :  1.2
5. data  :  1.02469507659596
6. secure  :  1.0198039027185568
7. php  :  1.0198039027185568
8. input  :  0.9165151389911681
9. could  :  0.9000000000000001
10. software  :  0.9000000000000001
11. question  :  0.9000000000000001
12. rail  :  0.9000000000000001
13. suhosin  :  0.9000000000000001
14. security  :  0.7810249675906655
15. web  :  0.7483314773547883
16. application  :  0.7483314773547883
17. available  :  0.6403124237432849
18. code  :  0.6403124237432849
19. .  :  0.6403124237432849
20. using  :  0.6403124237432849
21. (  :  0.6403124237432849
22. )  :  0.6403124237432849
23. use  :  0.6403124237432849
24. x  :  0.6403124237432849
25. http  :  0.6000000000000001
26. parsed  :  0.6000000000000001
27. mvc  :  0.6000000000000001
28. check  :  0.6000000000000001
29. nginx  :  0.6000000000000001
30. list  :  0.6000000000000001
31. ``  :  0.60000000000